In [ ]:
# Import python packages
import sys;
import json;

# Get JSON input parameters
y = json.loads(sys.argv[0])

# the result is a Python dictionary:
src_schema = "raw_layer"
external_stage_root_path = "@RAW_LAYER.EXTERNAL_AZUR_STAGE/Files"
bronze_table = "bronze_layer."  +y["bronze_table"]
stage_name = "@raw_layer.landing_internal_stage"
stage_path_suffix = "/" +y["bronze_table"].replace('_BRZ', '/')
pattern_file_name = "'" +y["pattern_file_name"]+ "'"
file_format = "bronze_layer." +y["file_format"]
on_error = "'" + y["on_error"]+ "'"
pattern_file_suffix = pattern_file_name.replace('.*.', '').upper()

internal_stage_asb_path = stage_name + stage_path_suffix
external_stage_abs_path = external_stage_root_path + stage_path_suffix

src_schema = "'" + src_schema + "'"
bronze_table = "'" + bronze_table + "'"
stage_path_suffix = "'" + stage_path_suffix + "'"
file_format = "'" + file_format + "'"

print("src_schema param: " + src_schema)
print("target_table param: " + bronze_table)
print("stage_name param: " + stage_name)
print("stage_path_suffix param: " + stage_path_suffix)
print("pattern_file_name param: " + pattern_file_name)
print("file_format param: " + file_format)
print("on_error param: " + on_error)
print("external_stage_root_path param: " + external_stage_root_path)

print("Builded internal_stage_abs_path: " + internal_stage_asb_path)
print("Builded external_stage_asb_path: " + external_stage_abs_path)



In [ ]:
copy files
into {{ internal_stage_asb_path }}
from {{ external_stage_abs_path }}
PATTERN = {{ pattern_file_name }};

In [ ]:
CALL bronze_layer.ingest_file_proc(
{{ src_schema }},
{{ bronze_table }},
{{ stage_name }},
{{ stage_path_suffix }},
{{ pattern_file_name }},
{{ file_format }},
{{ on_error }} 
);

In [ ]:
/*Check if all files are in "status" "LOADED". 
If NOT, raise an exception to fail the task*/

EXECUTE IMMEDIATE $$
DECLARE
  status_number NUMBER := (SELECT count(*) from {{ ingest_files_into_bronze_layer }} where "status" != 'LOADED') ;
  LOAD_NOT_COMPLETE_EXCEPTION EXCEPTION (-20002, 'FILES NOT LOADED COMPLETELY, CHECK MONITORING TABLE PLZ!');
BEGIN
  IF (status_number != 0) THEN
    RAISE LOAD_NOT_COMPLETE_EXCEPTION;
  END IF;
EXCEPTION
  WHEN OTHER THEN
    RAISE; -- Raise the same exception that you are handling.
END;
$$;